In [ ]:
import os
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from PIL import Image 
import numpy as np

def yolo_to_voc(yolo_dir, output_dir, image_dir, image_extension=".jpg"):
    """
    Converts YOLO annotation format to Pascal VOC format.

    :param yolo_dir: Directory containing YOLO .txt files
    :param output_dir: Directory to save VOC .xml files
    :param image_dir: Directory containing image files
    :param image_extension: Image file extension (default is .jpg)
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the class list; modify as needed based on actual requirements
    classes = ["fire", "smoke"]  # Ordered according to class_id in YOLO annotations

    for txt_file in os.listdir(yolo_dir):
        if not txt_file.endswith(".txt"):
            continue

        txt_path = os.path.join(yolo_dir, txt_file)
        xml_name = os.path.splitext(txt_file)[0] + ".xml"
        xml_path = os.path.join(output_dir, xml_name)
        image_name = os.path.splitext(txt_file)[0] + image_extension
        image_path = os.path.join(image_dir, image_name)

        # Use PIL to read the image and get its dimensions
        try:
            pil_img = Image.open(image_path)
            image = np.array(pil_img)
            height, width, depth = image.shape
        except Exception as e:
            print(f"Image not found or unable to read: {image_path}. Error: {e}")
            continue

        # Start building the XML
        annotation = ET.Element("annotation")
        ET.SubElement(annotation, "folder").text = os.path.basename(image_dir)
        ET.SubElement(annotation, "filename").text = image_name
        ET.SubElement(annotation, "path").text = image_path
        source = ET.SubElement(annotation, "source")
        ET.SubElement(source, "database").text = "Unknown"

        size = ET.SubElement(annotation, "size")
        ET.SubElement(size, "width").text = str(width)
        ET.SubElement(size, "height").text = str(height)
        ET.SubElement(size, "depth").text = str(depth)

        ET.SubElement(annotation, "segmented").text = "0"

        with open(txt_path, "r") as file:
            for line in file.readlines():
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Skipping invalid line in {txt_file}: {line}")
                    continue

                class_id, center_x, center_y, bbox_width, bbox_height = map(float, parts)

                
                class_id = int(class_id)
                if class_id < 0 or class_id >= len(classes):
                    print(f"Class ID {class_id} out of range in {txt_file}: {line}")
                    continue

                # Convert YOLO to VOC
                xmin = int((center_x - bbox_width / 2) * width)
                ymin = int((center_y - bbox_height / 2) * height)
                xmax = int((center_x + bbox_width / 2) * width)
                ymax = int((center_y + bbox_height / 2) * height)

                # Build object node
                obj = ET.SubElement(annotation, "object")
                ET.SubElement(obj, "name").text = classes[class_id]  
                ET.SubElement(obj, "pose").text = "Unspecified"
                ET.SubElement(obj, "truncated").text = "0"
                ET.SubElement(obj, "difficult").text = "0"

                bndbox = ET.SubElement(obj, "bndbox")
                ET.SubElement(bndbox, "xmin").text = str(max(0, xmin))
                ET.SubElement(bndbox, "ymin").text = str(max(0, ymin))
                ET.SubElement(bndbox, "xmax").text = str(min(width, xmax))
                ET.SubElement(bndbox, "ymax").text = str(min(height, ymax))

        # Format and write XML to file
        rough_string = ET.tostring(annotation, 'utf-8')
        reparsed = parseString(rough_string)
        pretty_xml = reparsed.toprettyxml(indent="\t")

        with open(xml_path, "w") as f:
            f.write(pretty_xml)

        print(f"Converted {txt_file} to {xml_path}")


# Usage example (please modify according to the actual path)
yolo_dir = r""  # Replace with YOLO annotation file directory
output_dir = r""  # Replace with the directory of the output XML file
image_dir = r""  # Replace with the directory of the image file

yolo_to_voc(yolo_dir, output_dir, image_dir, image_extension=".jpg")


Converted 00001.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00001.xml
Converted 00002.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00002.xml
Converted 00003.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00003.xml
Converted 00004.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00004.xml
Converted 00005.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00005.xml
Converted 00006.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00006.xml
Converted 00007.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00007.xml
Converted 00008.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00008.xml
Converted 00009.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00009.xml
Converted 00010.txt to G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00010.xml
Converted 00011.txt to G:\A-desktop\Expe

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
display_boxes.py

读取 Pascal VOC 格式的 XML 标注，并在对应图片上绘制边框和类别标签，
支持命令行参数指定输入/输出路径。
"""

import os
import sys
import argparse
import xml.etree.ElementTree as ET

import cv2
import matplotlib.pyplot as plt


def get_annotation_data(xml_path):
    """
    解析 Pascal VOC XML，返回图片尺寸和物体列表。
    每个物体是一个 dict，包含 'name' 和 'bbox'=(xmin, ymin, xmax, ymax)。
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size_node = root.find('size')
    width = int(size_node.find('width').text)
    height = int(size_node.find('height').text)

    objects = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.find('xmin').text))
        ymin = int(float(bndbox.find('ymin').text))
        xmax = int(float(bndbox.find('xmax').text))
        ymax = int(float(bndbox.find('ymax').text))
        objects.append({
            'name': name,
            'bbox': (xmin, ymin, xmax, ymax)
        })

    return (width, height), objects


def display_bounding_boxes(image_path, xml_path, save_path=None):
    """
    在图片上绘制所有标注框并显示（或保存）结果。
    如果 save_path 不为 None，则保存到磁盘并关闭窗口。
    """
    # 1. 读取图片
    img = cv2.imread(image_path)
    if img is None:
        print(f"错误：无法读取图片 {image_path}", file=sys.stderr)
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 2. 读取标注
    (w_xml, h_xml), objects = get_annotation_data(xml_path)
    h_img, w_img = img.shape[:2]
    if (w_xml, h_xml) != (w_img, h_img):
        print("警告：XML 中标注尺寸与实际图片尺寸不一致！", file=sys.stderr)
        print(f"  XML 尺寸: {w_xml}×{h_xml}，图片尺寸: {w_img}×{h_img}", file=sys.stderr)

    # 3. 绘制
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    ax = plt.gca()

    for obj in objects:
        xmin, ymin, xmax, ymax = obj['bbox']
        width = xmax - xmin
        height = ymax - ymin

        rect = plt.Rectangle(
            (xmin, ymin), width, height,
            fill=False, edgecolor='r', linewidth=2
        )
        ax.add_patch(rect)
        ax.text(
            xmin, ymin - 5, obj['name'],
            fontsize=12, color='red',
            verticalalignment='bottom',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1)
        )

    plt.axis('off')
    plt.tight_layout()

    # 4. 显示或保存
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
        print(f"已保存结果至 {save_path}")
        plt.close()
    else:
        plt.show()


def parse_args():
    parser = argparse.ArgumentParser(
        description="在 Pascal VOC 格式图片上绘制标注边框"
    )
    parser.add_argument(
        "-i", "--image", required=True,
        help="输入图片路径（JPG/PNG）"
    )
    parser.add_argument(
        "-x", "--xml", required=True,
        help="对应的 Pascal VOC XML 标注文件路径"
    )
    parser.add_argument(
        "-o", "--output", default=None,
        help="输出文件路径（如指定，则保存结果而不弹窗）"
    )
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    image_url = r"G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\images\00030.jpg"
    xml_url = r"G:\A-desktop\Experiment\datasets2\DM-FFSD\上传EasyDL\Annotations\00030.xml"
    
    # 检查图片和 XML 文件路径
    if not os.path.isfile(image_url):
        print(f"错误：找不到图片文件 {image_url}", file=sys.stderr)
        sys.exit(1)
    if not os.path.isfile(xml_url):
        print(f"错误：找不到 XML 文件 {xml_url}", file=sys.stderr)
        sys.exit(1)

    # 调用函数绘制边框
    display_bounding_boxes(image_url, xml_url, args.output)


usage: ipykernel_launcher.py [-h] -i IMAGE -x XML [-o OUTPUT]
ipykernel_launcher.py: error: the following arguments are required: -i/--image, -x/--xml


SystemExit: 2